# 🔬 Claude Tools - Hybrid Processor

**Best of both worlds**: Auto-runs when opened, but keeps cell structure for debugging

- ✅ **Auto-executes** when notebook opens
- ✅ **Cell structure** for debugging
- ✅ **Live monitoring** dashboard
- ✅ **Interactive analysis** tools

In [ ]:
#@title 🚀 Auto-Start Setup { display-mode: "form" }
#@markdown This cell auto-runs when the notebook opens

import IPython
from IPython.display import Javascript

# Auto-run all cells after 3 seconds
display(Javascript('''
setTimeout(function() {
    console.log("Auto-starting Claude Tools processor...");
    // Check if we're in Colab
    if (window.colab) {
        // Find and click "Run all" button
        document.querySelector('[command="runall"]').click();
    }
}, 3000);
'''))

print("⏳ Auto-starting in 3 seconds...")
print("💡 You can also run cells individually for debugging")

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted")

In [ ]:
#@title 📦 Dependencies & Configuration { display-mode: "form" }

#@markdown Folder ID for Claude Tools:
FOLDER_ID = "1tzHn4J3QntSLJlJNXcNJe3cLdILGEb3Z" #@param {type:"string"}

#@markdown Processing settings:
AUTO_RUN_DURATION = 3600 #@param {type:"slider", min:60, max:3600, step:60}
POLL_INTERVAL = 3 #@param {type:"slider", min:1, max:10, step:1}
DEBUG_MODE = True #@param {type:"boolean"}

!pip install -q google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pandas matplotlib

import os
import json
import time
import traceback
from datetime import datetime
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
import io
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output
import threading

print("✅ Dependencies loaded")
print(f"📁 Folder: {FOLDER_ID}")
print(f"⏱️  Duration: {AUTO_RUN_DURATION}s")
print(f"🔄 Poll: {POLL_INTERVAL}s")

In [ ]:
# Initialize services
auth.authenticate_user()
drive_service = build('drive', 'v3')
print("✅ Authenticated with Google Drive")

In [ ]:
#@title 🤖 Hybrid Processor Class { display-mode: "form" }

class HybridProcessor:
    """Processor with both auto-run and debug features"""
    
    def __init__(self, folder_id, drive_service, debug=True):
        self.folder_id = folder_id
        self.drive_service = drive_service
        self.processed_requests = set()
        self.execution_history = []
        self.debug_mode = debug
        self.stats = {
            'processed': 0,
            'errors': 0,
            'total_time': 0,
            'start_time': time.time()
        }
        self.running = False
        
    def get_pending_requests(self):
        """Get unprocessed command files"""
        try:
            query = f"'{self.folder_id}' in parents and name contains 'command_' and trashed=false"
            results = self.drive_service.files().list(
                q=query,
                fields="files(id, name, createdTime)",
                orderBy="createdTime"
            ).execute()
            
            requests = []
            for file in results.get('files', []):
                if file['id'] not in self.processed_requests:
                    requests.append(file)
                    
            return requests
        except Exception as e:
            if self.debug_mode:
                print(f"❌ Error listing requests: {e}")
            return []
    
    def process_request(self, request_file):
        """Process a single request"""
        try:
            # Read request
            content = self.drive_service.files().get_media(fileId=request_file['id']).execute()
            request_data = json.loads(content.decode('utf-8'))
            
            # Extract command ID
            command_id = request_file['name'].replace('command_', '').replace('.json', '')
            
            if self.debug_mode:
                print(f"\n📋 Processing: {command_id}")
            
            # Execute code
            start_time = time.time()
            result = self.execute_code(request_data.get('code', ''))
            execution_time = time.time() - start_time
            
            result['execution_time'] = execution_time
            
            # Write response
            self.write_response(command_id, result)
            
            # Update stats
            self.stats['processed'] += 1
            self.stats['total_time'] += execution_time
            
            if result['status'] == 'error':
                self.stats['errors'] += 1
                
            # Mark as processed
            self.processed_requests.add(request_file['id'])
            
            # Store in history
            self.execution_history.append({
                'command_id': command_id,
                'status': result['status'],
                'execution_time': execution_time,
                'timestamp': time.time()
            })
            
        except Exception as e:
            if self.debug_mode:
                print(f"❌ Error processing request: {e}")
            self.stats['errors'] += 1
    
    def execute_code(self, code):
        """Execute code safely"""
        from io import StringIO
        import sys
        
        old_stdout = sys.stdout
        sys.stdout = StringIO()
        
        try:
            namespace = {'__name__': '__main__'}
            exec(code, namespace)
            output = sys.stdout.getvalue()
            
            return {
                'status': 'success',
                'output': output,
                'timestamp': time.time()
            }
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e),
                'traceback': traceback.format_exc(),
                'timestamp': time.time()
            }
        finally:
            sys.stdout = old_stdout
    
    def write_response(self, command_id, response_data):
        """Write response to Drive"""
        try:
            response_name = f'result_{command_id}.json'
            
            file_metadata = {
                'name': response_name,
                'parents': [self.folder_id]
            }
            
            media = MediaIoBaseUpload(
                io.BytesIO(json.dumps(response_data, indent=2).encode('utf-8')),
                mimetype='application/json'
            )
            
            self.drive_service.files().create(
                body=file_metadata,
                media_body=media
            ).execute()
            
            if self.debug_mode:
                print(f"✅ Response written: {response_name}")
                
        except Exception as e:
            if self.debug_mode:
                print(f"❌ Error writing response: {e}")
    
    def run_auto(self, duration=3600, poll_interval=3):
        """Auto-run processor"""
        self.running = True
        start_time = time.time()
        
        while self.running and (time.time() - start_time < duration):
            try:
                requests = self.get_pending_requests()
                
                if requests:
                    for req in requests:
                        self.process_request(req)
                
                time.sleep(poll_interval)
                
            except KeyboardInterrupt:
                self.running = False
                break
            except Exception as e:
                if self.debug_mode:
                    print(f"❌ Error in main loop: {e}")
                time.sleep(5)
        
        self.running = False
    
    def get_status(self):
        """Get current processor status"""
        elapsed = time.time() - self.stats['start_time']
        
        return {
            'running': self.running,
            'processed': self.stats['processed'],
            'errors': self.stats['errors'],
            'success_rate': (self.stats['processed'] - self.stats['errors']) / max(self.stats['processed'], 1),
            'avg_time': self.stats['total_time'] / max(self.stats['processed'], 1),
            'elapsed': elapsed,
            'pending': len(self.get_pending_requests())
        }

# Create processor instance
processor = HybridProcessor(FOLDER_ID, drive_service, debug=DEBUG_MODE)
print("✅ Hybrid processor initialized")

In [ ]:
#@title 📊 Live Dashboard { display-mode: "form" }
#@markdown Real-time monitoring dashboard

def create_dashboard():
    """Create HTML dashboard"""
    status = processor.get_status()
    
    dashboard_html = f"""
    <div style="font-family: monospace; background: #f0f0f0; padding: 20px; border-radius: 10px;">
        <h2 style="color: #2ecc71;">🤖 Claude Tools Processor</h2>
        <hr>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
            <div>
                <h3>📊 Statistics</h3>
                <p>⚡ Status: <strong>{"Running" if status['running'] else "Stopped"}</strong></p>
                <p>✅ Processed: <strong>{status['processed']}</strong></p>
                <p>❌ Errors: <strong>{status['errors']}</strong></p>
                <p>📈 Success Rate: <strong>{status['success_rate']:.1%}</strong></p>
            </div>
            <div>
                <h3>⏱️ Performance</h3>
                <p>⏳ Runtime: <strong>{int(status['elapsed'])}s</strong></p>
                <p>⚡ Avg Time: <strong>{status['avg_time']:.3f}s</strong></p>
                <p>📨 Pending: <strong>{status['pending']}</strong></p>
                <p>🔄 Poll Rate: <strong>{POLL_INTERVAL}s</strong></p>
            </div>
        </div>
    </div>
    """
    
    return HTML(dashboard_html)

# Display initial dashboard
display(create_dashboard())

In [ ]:
#@title 🚀 Start Auto-Processor { display-mode: "form" }
#@markdown Starts processing in background thread

# Start processor in background thread
def run_processor_thread():
    processor.run_auto(duration=AUTO_RUN_DURATION, poll_interval=POLL_INTERVAL)

# Start thread
processor_thread = threading.Thread(target=run_processor_thread)
processor_thread.daemon = True
processor_thread.start()

print(f"✅ Auto-processor started!")
print(f"⏱️  Will run for {AUTO_RUN_DURATION//60} minutes")
print(f"💡 Continue to next cells for monitoring and debugging")

# Update dashboard every 5 seconds
import time
for i in range(12):  # Show for 1 minute
    time.sleep(5)
    clear_output(wait=True)
    display(create_dashboard())
    
print("\n📊 Dashboard updated. Continue to next cells for more tools.")

## 🔍 Debug Tools

The cells below provide interactive debugging while the processor runs:

In [ ]:
# Check current status
status = processor.get_status()
print("📊 Current Status:")
for key, value in status.items():
    print(f"   {key}: {value}")

In [ ]:
# View execution history
if processor.execution_history:
    df = pd.DataFrame(processor.execution_history)
    print("📜 Recent Executions:")
    display(df.tail(10))
    
    # Plot execution times
    if len(df) > 1:
        plt.figure(figsize=(10, 4))
        plt.plot(df.index, df['execution_time'], 'b-o')
        plt.xlabel('Execution Number')
        plt.ylabel('Time (seconds)')
        plt.title('Execution Performance')
        plt.grid(True, alpha=0.3)
        plt.show()
else:
    print("No execution history yet")

In [ ]:
# List pending requests
pending = processor.get_pending_requests()
print(f"📨 {len(pending)} pending requests:")
for req in pending[:5]:
    print(f"   - {req['name']}")

In [ ]:
# Create a test request
test_code = f'''
print("🧪 Debug test at {datetime.now()}")
import numpy as np
result = np.random.rand(10).mean()
print(f"Random mean: {{result:.4f}}")
'''

# Create request file
request_data = {
    'type': 'execute',
    'code': test_code,
    'timestamp': time.time(),
    'source': 'debug_cell'
}

command_id = f"debug_{int(time.time())}"
file_name = f"command_{command_id}.json"

file_metadata = {
    'name': file_name,
    'parents': [FOLDER_ID]
}

media = MediaIoBaseUpload(
    io.BytesIO(json.dumps(request_data, indent=2).encode('utf-8')),
    mimetype='application/json'
)

file = drive_service.files().create(
    body=file_metadata,
    media_body=media
).execute()

print(f"✅ Created test request: {file_name}")
print("⏳ It will be processed automatically...")

In [ ]:
# Stop processor if needed
# processor.running = False
# print("🛑 Processor stopped")

## 📊 Performance Analysis

Analyze the processor's performance after it has run:

In [ ]:
# Final summary
if processor.execution_history:
    df = pd.DataFrame(processor.execution_history)
    
    print("📊 Session Summary:")
    print(f"Total executions: {len(df)}")
    print(f"Success rate: {(df['status'] == 'success').mean():.1%}")
    print(f"Average time: {df['execution_time'].mean():.3f}s")
    print(f"Total runtime: {int(time.time() - processor.stats['start_time'])}s")
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Execution timeline
    axes[0, 0].plot(df['execution_time'], 'b-')
    axes[0, 0].set_title('Execution Time per Request')
    axes[0, 0].set_ylabel('Time (s)')
    
    # Success/Error pie
    status_counts = df['status'].value_counts()
    axes[0, 1].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%')
    axes[0, 1].set_title('Success vs Error Rate')
    
    # Execution time histogram
    axes[1, 0].hist(df['execution_time'], bins=20, alpha=0.7)
    axes[1, 0].set_title('Execution Time Distribution')
    axes[1, 0].set_xlabel('Time (s)')
    
    # Cumulative requests
    axes[1, 1].plot(range(len(df)), range(len(df)), 'g-')
    axes[1, 1].set_title('Cumulative Requests Processed')
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].set_ylabel('Total Requests')
    
    plt.tight_layout()
    plt.show()
else:
    print("No execution data to analyze")

## 🎯 Summary

This **Hybrid Processor** provides:

1. **Auto-execution** - Starts processing automatically when opened
2. **Cell structure** - Each component in separate cells for debugging
3. **Background processing** - Runs in thread while you debug
4. **Live monitoring** - Dashboard updates in real-time
5. **Interactive tools** - Test and debug while running

Perfect for:
- **Production use** - Auto-runs for hands-free operation
- **Development** - Debug individual components
- **Analysis** - Monitor and analyze performance